In [2]:
%load_ext autoreload
%autoreload 2
import os
os.chdir("/projects/cft_vlm/finetune/")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
import json
import tempfile
from transformers import Qwen2_5_VLProcessor

def test_qwen_processor():
  # 1. Load Qwen2.5 VL processor from Hugging Face
  proc1 = Qwen2_5_VLProcessor.from_pretrained("Qwen/Qwen2.5-VL-3B-Instruct", trust_remote_code=True)
  
  # 2. Monkey patch an attribute (e.g., add a new attribute)
  proc1.monkey_patch_attr = "patched_value"
  
  # 3. Save the processor configuration to a JSON file
  with tempfile.TemporaryDirectory() as tmpdir:
    json_path = f"{tmpdir}/processor.json"
    proc1.to_json_file(json_path)
    with open(json_path, 'r') as f:
      d1 = json.load(f)
    # 4. Load a processor from that JSON file
    proc_from_file = Qwen2_5_VLProcessor.from_args_and_dict(processor_dict=d1)
  
  # 5. Load a fresh processor from Hugging Face and apply the same monkey patch
  proc2 = Qwen2_5_VLProcessor.from_pretrained("Qwen/Qwen2.5-VL-3B-Instruct")
  proc2.monkey_patch_attr = "patched_value"
  
  d1 = proc_from_file.to_dict()
  d2 = proc2.to_dict()
  assert "monkey_patch_attr" in d1, "Monkey patch attribute not found in the first processor."
  assert "monkey_patch_attr" in d2, "Monkey patch attribute not found in the second processor."
  assert proc_from_file.image_processor.to_dict() == proc2.image_processor.to_dict(), "Image processors do not match."
  assert proc_from_file.video_processor.to_dict() == proc2.video_processor.to_dict(), "Video processors do not match."

# Running the test
test_qwen_processor()
print("Test passed!")

TypeError: ProcessorMixin.from_args_and_dict() missing 1 required positional argument: 'args'